# Feature Engineering

For some definitions, handling outliers and missing values, scaling, and encoding may be considered feature engineering. Here we'll draw a distinction between data preparation, data preprocessing, and feature engineering.

- **data preparation**: the basic data cleaning necessary to get our data ready for exploration/analysis, e.g. correcting data types, fixing typos
- **data preprocessing**: further data transformation done for the sake of modeling, as oppsoed to exploration/analysis, e.g. scaling, imputing, encoding
- **feature engineering**: adding, combining, or removing features; usually with the help of domain knowledge

Feature engineering can happen as part of data exploration or modeling, and engineered featured are also commonly explored.

Some examples of feature engineering by this definition:

- domain-based conversion (example: farenheit to celsius, BMI calculation, log transformation)
- domain based cutoffs (example: age >= 18 = is_adult; also dates)

- add / subtract (example: zillow dataset: beds + baths = room_count; total_sqft - 200 * bedrooms - 40 * bathrooms = living_area)
- combine as booleans as a count (example: telco_churn: streaming + backups + ...  = service_count)
- multiply / divide (example: tips dataset: total_bill / size = price_per_person)
- ratios (example: tips dataset: tip / total_bill = tip percentage)

Simplify!

- categorical with many unique values to top 3 + "Other"
- categorical to boolean: pool count -> has pool
- continous -> categorical via binning (aka quantization or discretization) (example: income -> high, medium, low earner)

In this lesson we'll cover some *automated* **feature selection** methods, that is, methods for determining which features are the most important.

- SelectKBest
- Recursive Feature Elimination
- Sequential Feature Selection

## Setup

In [2]:
import numpy as np
import pandas as pd
import wrangle
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector

In [15]:
# df = wrangle.wrangle_grades()

df = pd.read_clipboard()
df.head()
df.isna().sum()
df = df.dropna()
df.isna().sum()
df = df.drop(columns='student_id')

In [16]:
df

,exam1,exam2,exam3,final_grade
0,100.0,90,95.0,96
1,98.0,93,96.0,95
2,85.0,83,87.0,87
3,83.0,80,86.0,85
4,93.0,90,96.0,97
5,79.0,70,85.0,81
6,92.0,89,94.0,93
7,73.0,70,75.0,76
8,70.0,65,78.0,77
9,62.0,70,79.0,70


In [17]:

from sklearn.model_selection import train_test_split

train_validate, test = train_test_split(df)
train, validate = train_test_split(train_validate)



In [21]:
X_train = train.drop(columns='final_grade')
y_train = train['final_grade']

## Select K Best

- looks at each feature in isolation against the target based on correlation
- fastest of all approaches covered in this lesson
- doesn't consider feature interactions
- After fitting: `.scores_`, `.pvalues_`, `.get_support()`, and `.transform`

In [22]:
# make the thing

kbest = SelectKBest(f_regression, k=2)

# fit the thing

_ = kbest.fit(X_train, y_train)

In [24]:
# statistical f-value:

kbest.scores_

#p value: 

kbest.pvalues_

array([8.42513719e-42, 1.67194226e-22, 2.57812159e-32])

In [25]:
# get-support() will output a boolean mask to tell me which features were selected
# we can apply this mask to the columns in our original dataframe

kbest.get_support()

array([ True, False,  True])

In [27]:
train.head()

,exam1,exam2,exam3,final_grade
70,79.0,70,85.0,81
18,79.0,70,85.0,81
94,83.0,80,86.0,85
2,85.0,83,87.0,87
25,70.0,75,78.0,72


In [28]:
X_train.iloc[:, kbest.get_support()]

,exam1,exam3
70,79.0,85.0
18,79.0,85.0
94,83.0,86.0
2,85.0,87.0
25,70.0,78.0
100,62.0,79.0
78,100.0,95.0
51,70.0,78.0
43,93.0,96.0
87,62.0,79.0


In [30]:
# can use this as a mask to keep the DF as a DF

X_train.columns[kbest.get_support()]

Index(['exam1', 'exam3'], dtype='object')

In [44]:
# kbest transform will convert our information to the selected
# feature subspace
# ****buuuuuut, its just a numpy array
kbest.transform(X_train)

array([[ 79.,  85.],
       [ 79.,  85.],
       [ 83.,  86.],
       [ 85.,  87.],
       [ 70.,  78.],
       [ 62.,  79.],
       [100.,  95.],
       [ 70.,  78.],
       [ 93.,  96.],
       [ 62.,  79.],
       [ 83.,  86.],
       [ 73.,  75.],
       [ 70.,  78.],
       [ 70.,  78.],
       [ 70.,  78.],
       [ 70.,  78.],
       [ 70.,  78.],
       [ 93.,  96.],
       [ 58.,  70.],
       [ 79.,  85.],
       [ 70.,  78.],
       [ 98.,  96.],
       [ 57.,  75.],
       [ 83.,  86.],
       [ 83.,  86.],
       [ 70.,  78.],
       [ 57.,  75.],
       [ 62.,  79.],
       [ 93.,  96.],
       [100.,  95.],
       [ 98.,  96.],
       [ 85.,  87.],
       [ 83.,  86.],
       [ 92.,  94.],
       [ 85.,  87.],
       [ 98.,  96.],
       [ 58.,  70.],
       [ 70.,  78.],
       [ 93.,  96.],
       [ 57.,  75.],
       [ 73.,  75.],
       [ 62.,  79.],
       [ 79.,  85.],
       [ 98.,  96.],
       [ 57.,  75.],
       [ 70.,  78.],
       [ 58.,  70.],
       [ 79.,

## RFE

- Recursive Feature Elimination
- Progressively eliminate features based on importance to the model
- Requires a model with either a `.coef_` or `.feature_importances_` property
- After fitting: `.ranking_`, `.get_support()`, and `.transform()`

In [33]:
from sklearn.linear_model import LinearRegression

In [35]:
model = LinearRegression()

In [36]:
rfe = RFE(model, n_features_to_select=2)

In [37]:
rfe.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=2)

In [38]:
rfe.ranking_

array([1, 2, 1])

In [40]:
rfe.get_support()

array([ True, False,  True])

In [41]:
pd.DataFrame({
    'rfe_ranking': rfe.ranking_
}, index=X_train.columns)

,rfe_ranking
exam1,1
exam2,2
exam3,1


In [45]:
pd.DataFrame(rfe.transform(X_train), columns=X_train.columns[rfe.get_support()], index=X_train.index)

,exam1,exam3
70,79.0,85.0
18,79.0,85.0
94,83.0,86.0
2,85.0,87.0
25,70.0,78.0
100,62.0,79.0
78,100.0,95.0
51,70.0,78.0
43,93.0,96.0
87,62.0,79.0


## Sequential Feature Selector

- progressively adds features based on cross validated model performance
- forwards: start with 0, add the best additional feature until you have the desired number
- backwards: start with all features, remove the worst performing until you have the desired number
- After fitting: `.support_`, `.transform`

In [47]:
model = LinearRegression()

sfs = SequentialFeatureSelector(model, n_features_to_select=2)

sfs.fit(X_train, y_train)

SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=2)

In [48]:
sfs.transform(X_train)

array([[ 79.,  85.],
       [ 79.,  85.],
       [ 83.,  86.],
       [ 85.,  87.],
       [ 70.,  78.],
       [ 62.,  79.],
       [100.,  95.],
       [ 70.,  78.],
       [ 93.,  96.],
       [ 62.,  79.],
       [ 83.,  86.],
       [ 73.,  75.],
       [ 70.,  78.],
       [ 70.,  78.],
       [ 70.,  78.],
       [ 70.,  78.],
       [ 70.,  78.],
       [ 93.,  96.],
       [ 58.,  70.],
       [ 79.,  85.],
       [ 70.,  78.],
       [ 98.,  96.],
       [ 57.,  75.],
       [ 83.,  86.],
       [ 83.,  86.],
       [ 70.,  78.],
       [ 57.,  75.],
       [ 62.,  79.],
       [ 93.,  96.],
       [100.,  95.],
       [ 98.,  96.],
       [ 85.,  87.],
       [ 83.,  86.],
       [ 92.,  94.],
       [ 85.,  87.],
       [ 98.,  96.],
       [ 58.,  70.],
       [ 70.,  78.],
       [ 93.,  96.],
       [ 57.,  75.],
       [ 73.,  75.],
       [ 62.,  79.],
       [ 79.,  85.],
       [ 98.,  96.],
       [ 57.,  75.],
       [ 70.,  78.],
       [ 58.,  70.],
       [ 79.,

In [49]:
sfs.get_support()

array([ True, False,  True])

In [50]:
X_train.columns[sfs.get_support()]

Index(['exam1', 'exam3'], dtype='object')

## Conclusion

- Simpler models handle change + variability better
- Use RFE to narrow down your features and find the best ones, if your dataset is large (> 1GB; `df.info()`) use select k best instead
- Remember: feature engineering is much more than feature selection!